## Wolframbeta ##
---
### 구현된 기능 ###
- 사칙연산
    두 실수의 사칙연산
    괄호를 포함한 여러 실수의 사칙연산
    괄호, 곱셈, 덧셈을 포함한 여러 실수의 연산자 우선순위를 따르는 사칙연산
- 거듭제곱
    거듭제곱의 우선순위는 괄호 > 거듭제곱 > 곱셈, 나눗셈
- 연산 트리 생성
- 일반함수의 토큰화


### TODO ###
- pi, e 등의 수학 상수 지원(부동소수점 실수 근사값)
- 초등함수 (대수함수, 삼각함수, 지수함수, 로그함수)
- 합성함수
- 함수 정의
- 다항식의 계산
- 함수값의 계산
- 일변수 함수의 미분
- 다변수 함수의 정의
- 다변수 함수의 편미분
- 미분가능성 판단

- GUI 인터페이스
- 함수 출력, 함수 그래프 출력

In [1]:
from wolframbeta.utils import *

In [288]:
CONST_KEY = "$const"
COEFF_KEY = "$coeff"
SUCCESS_CODE = "SUCCESS"

In [179]:
class Variable(str):
    pass

In [263]:
class ExprDict(dict):
    def __init__(self, *args):
        if isinstance(*args, float) or isinstance(*args, int):
            super(self.__class__, self).__init__()
            self[CONST_KEY] = args[0]
        else:
            if len(args) > 0 and isinstance(args[0], TermDict):
                super(self.__class__, self).__init__()
                if args[0].is_constant():
                    self[CONST_KEY] = args[0][COEFF_KEY]
                else:
                    self[args[0]] = args[0][COEFF_KEY]
                    self[CONST_KEY] = 0.0
            else:
                super(self.__class__, self).__init__(*args)
                if CONST_KEY not in self.keys():
                    self[CONST_KEY] = 0.0

    def __add__(self, other):
        ret_dict = None
        if isinstance(other, int) or isinstance(other, float):
            ret_dict = ExprDict(self)
            ret_dict[CONST_KEY] += other
            
        elif isinstance(other, TermDict):
            ret_dict = ExprDict(self)
            if other.is_constant():
                const = other.get_coefficient()
                ret_dict = self + const
            else:
                print( other, ret_dict )
                if other in ret_dict.keys():
                    print("in")
                    ret_dict[other] += other.get_coefficient()
                    if ret_dict[other] == 0:
                        ret_dict.pop(other, None)
                else:
                    print("not in")
                    ret_dict[other] = other.get_coefficient()

        elif isinstance(other, ExprDict):
            ret_dict = ExprDict(self)
            for term, coeff in other.items():
                if term == CONST_KEY:
                    ret_dict[CONST_KEY] = ret_dict[CONST_KEY] + other[CONST_KEY]
                else:
                    ret_dict = ret_dict + term
        
        if ret_dict.has_one_term():
            ret_dict = ret_dict.get_one_term()
            
        return ret_dict
    
    def __sub__(self, other):
        return self + -other
    
    def __neg__(self, other):
        ret_dict = ExprDict(self)
        for term, coeff in ret_dict.items():
            ret_dict[term] = -coeff
        return ret_dict
    
    def is_constant(self):
        return len(self) == 1
    
    def has_one_term(self):
        if self.get_constant() == 0:
            if len(self) == 2:
                return True
            else:
                return False
        else:
            return len(self) == 1
    
    def get_constant(self):
        return self[CONST_KEY]
    
    def get_one_term(self):
        """
        return (one term, its coefficient)
        """
        if self.is_constant():
            return TermDict(self.get_constant()), 1.0
        else:
            for term, coeff in self.items():
                if term == CONST_KEY:
                    continue
                else:
                    return term, coeff

In [319]:
class TermDict(dict):
    def __init__(self, *args):
        if isinstance(*args, float) or isinstance(*args, int):
            super(self.__class__, self).__init__()
            self[COEFF_KEY] = args[0]
        else:
            super(self.__class__, self).__init__(*args)
            if len(args) > 0 and isinstance(args[0], TermDict):
                self[COEFF_KEY] = args[0].get_coefficient()
            else:
                self[COEFF_KEY] = 1.0

    def __add__(self, other):

        ret_dict = None
        if isinstance(other, ExprDict):
            return other + self
        elif isinstance(other, TermDict):
            if str(self) == str(other):
                if self[COEFF_KEY] + other.get_coefficient() == 0:
                    ret_dict = TermDict(0)
                else:
                    ret_dict = TermDict(self)
                    ret_dict[COEFF_KEY] += other.get_coefficient()
            else:
                ret_dict = ExprDict(other)
                ret_dict = ret_dict + self
        elif isinstance(other, float) or isinstance(other, int):
            ret_dict = ExprDict(other)
            ret_dict = ret_dict + self

        if ret_dict is None:
            raise_error("Error : TermDict + unknown")
            ret_dict = TermDict()

        return ret_dict

    def __radd__(self, other):
        return self + other

    def __sub__(self, other):
        return self + -other

    def __rsub__(self, other):
        return other + -self

    def __mul__(self, other):
        if isinstance(other, TermDict):
            ret_dict = TermDict(self)
            ret_dict[COEFF_KEY] *= other.get_coefficient()
            for factor, power in other.items():
                if factor == COEFF_KEY:
                    continue
                if factor in ret_dict.keys():
                    ret_dict[factor] = ret_dict[factor] + power
                    if ret_dict[factor] == 0:
                        ret_dict.pop(factor, None)
                else:
                    ret_dict[factor] = power
            return ret_dict

        elif isinstance(other, ExprDict):
            return ExprDict * self

        elif isinstance(other, float) or isinstance(other, int):
            if other == 0:
                return TermDict(0)
            ret_dict = TermDict(self)
            ret_dict[COEFF_KEY] *= float(other)
            return ret_dict

        else:
            raise_error("Not implemented : termdict * unknown")

    def __rmul__(self, other):
        return self * other

    def __truediv__(self, other):
        if isinstance(other, TermDict):
            ret_dict = TermDict(self)
            ret_dict[COEFF_KEY] /= other.get_coefficient()
            for factor, power in other.items():
                if factor == COEFF_KEY:
                    continue
                if factor in ret_dict.keys():
                    ret_dict[factor] = ret_dict[factor] - other[factor]
                    if ret_dict[factor] == 0:
                        ret_dict.pop(factor, None)
                else:
                    ret_dict[factor] = -other[factor]
            return ret_dict
        elif isinstance(other, int) or isinstance(other, float):
            ret_dict = TermDict(self)
            ret_dict[COEFF_KEY] /= float(other)
            return ret_dict
        elif isinstance(other, ExprDict):
            return other.__rtruediv__(self)
        else:
            raise_error("Not implemented : TermDict / Unknown")

    def __rtruediv__(self, other):
        if isinstance(other, float) or isinstance(other, int):
            ret_dict = TermDict(self)
            for factor, power in self.items():
                if factor == COEFF_KEY:
                    ret_dict[COEFF_KEY] = 1/ret_dict.get_coefficient()
                else:
                    ret_dict[factor] = -ret_dict[factor]
            return ret_dict
        else:
            raise_error("Not implemented : unknown / termdict")

    def __pow__(self, power, modulo=None):
        if isinstance(power, float):
            if power == 0:
                return TermDict(1)
            ret_dict = TermDict(self)
            for key in ret_dict.keys():
                if key == COEFF_KEY:
                    ret_dict[key] = ret_dict[key]**power
                else:
                    ret_dict[key] *= power
            return ret_dict
        elif isinstance(power, int):
            return self ** float(power)
        elif isinstance(power, TermDict):
            if power.is_constant():
                return self ** power.get_coefficient()
            else:
                ret_dict = TermDict(self)
                for factor, po in self.items():
                    if factor == COEFF_KEY:
                        if self.get_coefficient() != 1:
                            if power.get_coefficient() == 1:
                                coff_td = TermDict(self.get_coefficient())
                                ret_dict[coff_td] = power
                            else:
                                coff_td = TermDict(self.get_coefficient() ** power.get_coefficient())
                                ret_dict[coff_td] = power / power.get_coefficient()
                        ret_dict[COEFF_KEY] = 1
                        
                    else:
                        ret_dict[factor] = ret_dict[factor] * power
                return ret_dict
            pass
        elif isinstance(power, ExprDict):
            pass

    def __rpow__(self, other):
        if isinstance(other, float) or isinstance(other, int):
            new_const = TermDict(other)
            return new_const ** self

    def __neg__(self):
        ret_dict = TermDict(self)
        ret_dict[COEFF_KEY] *= -1.0
        return ret_dict

    def __str__(self):
        if self.is_constant():
            ret_str = str(self.get_coefficient())
        else:
            if self.get_coefficient() == 1:
                ret_str = self.get_str_without_coeff()
            elif self.get_coefficient() == -1:
                ret_str = "-" + self.get_str_without_coeff()
            else:
                ret_str = str(self.get_coefficient()) + "*" + self.get_str_without_coeff()
        return ret_str

    def __eq__(self, other):
        if isinstance(other, TermDict):
            return hash(self) == hash(other)
        return super(self.__class__, self).__eq__(other)
    
    def __lt__(self, other):
        if isinstance(other, str):
            return False
        elif isinstance(other, TermDict):
            if len(self) != len(other):
                return len(self) > len(other)
            else:
                self_dim = 0
                other_dim = 0
                for factor, power in other.items():
                    other_dim = other_dim + power
                for factor, power in self.items():
                    self_dim = self_dim + power

                if (isinstance(self_dim, float) or isinstance(self_dim, int)) and (
                    isinstance(other_dim, float) or isinstance(other_dim, int)):
                    return self_dim < other_dim
                else:
                    return str(self) > str(other)
        
    def __hash__(self):
        return hash(self.get_str_without_coeff())

    def is_constant(self):
        return len(self) == 1

    def has_one_term(self):
        return True
    
    def get_one_term(self):
        return self
    
    def get_coefficient(self):
        return strip_float(self[COEFF_KEY])

    def get_constant(self):
        return strip_float(self[COEFF_KEY])
    
    def calculate_variable(self, variable_dict):
        """
        ret_dict, ret_code(계산 가능한지? SUCCESS이면 가능)
        """
        ret_dict = TermDict(1)
        ret_code = "SUCCESS"
        for factor, power in self.items():
            if factor == COEFF_KEY:
                ret_dict[COEFF_KEY] *= power
                continue
            elif isinstance(factor, Variable):
                if factor in variable_dict.keys():
                    factor_calculated = variable_dict[factor]
                else:
                    factor_calculated = factor
            else:
                factor_calculated, fac_code = factor.calculate_variable(variable_dict)
            
            if isinstance(power, int) or isinstance(power, float):
                power_calculated = power
            elif isinstance(power, Variable):
                if power in variable_dict.keys():
                    power_calculated = variable_dict[power]
                else:
                    power_calculated = power
            else:
                power_calculated, return_code = power.calculate_variable(variable_dict)
                if return_code != SUCCESS_CODE:
                    ret_code = return_code
            
            if isinstance(power_calculated, float) or isinstance(power_calculated, int):
                # 계산한 지수가 상수
                if power_calculated == 0:
                    continue
                    
                # factor_calculated이 될 수 있는 것 : 항(expd, td), 변수(variable), 숫자
                if isinstance(factor_calculated, float) or isinstance(factor_calculated, int):
                    ret_dict[COEFF_KEY] *= factor_calculated ** power_calculated
                    pass
                elif isinstance(factor_calculated, Variable):
                    ret_dict[factor_calculated] = power_calculated
                    pass
                else:
                    pass
                
                    
            
        return ret_dict, ret_code
                
            
                

    def get_str_without_coeff(self):
        if self.is_constant():
            return CONST_KEY
        ret_str = ""
        write_coeff = 0
        for i, (factor, _power) in enumerate(sorted(self.items())):
            if factor == COEFF_KEY:
                write_coeff = 1
                continue

            if isinstance(_power, float) and _power.is_integer():
                power = int(_power)
            else:
                power = _power

            if i-write_coeff > 0 and i-write_coeff < len(self.keys()):
                ret_str += "*"
            if factor == COEFF_KEY:
                ret_str += str(power)
            else:
                if isinstance(power, float) or isinstance(power, int):
                    if power == 1:
                        ret_str += str(factor)
                    elif power == -1:
                        ret_str += '1' + '/(' + str(factor) + ')'
                    elif power < 0:
                        ret_str += '1' + '/(' + str(factor) + '^'  + str(-power)+')'
                    else:
                        ret_str += str(factor) + '^' + str(power)
                else:
                    ret_str += str(factor) + '^(' + str(power) +')'
            if ret_str[0] == '*':
                ret_str = ret_str[1:]
        return ret_str


td + td
td - td
td * td
td / td
td + 1
td - 1
td * 2
td / 2
td ** td

ed + td
ed - td
ed * td
ed / td
ed ** td


ed + ed
ed - ed
ed * ed
ed / ed
ed ** ed




In [308]:
import random

In [309]:
# coeffs = [-3., -3, -2., -2, -1., -1, -0.5, 0., 0, 0.5, 1., 1, 2., 2, 3., 3]
coeffs = [-3., -3, -2., -2, -1., -1, -0.5, 0.5, 1., 1, 2., 2, 3., 3]
vs = ['x', 'y', 'z']

def get_random_td():
    random.shuffle(coeffs)
    td = TermDict(coeffs[0])
    
    num_var = random.randint(0,3)
    random.shuffle(vs)
    for i in range(num_var):
        random.shuffle(coeffs)
#         td[vs[i]] = float(int(random.random() * 10))/10
        td[vs[i]] = coeffs[0]
    
    return td
    
    




In [312]:
td1 = get_random_td()
td2 = get_random_td()
td1, td2, td1 ** td2

({'$coeff': -3.0, 'x': -2, 'y': 3, 'z': -1.0},
 {'$coeff': -1},
 {'$coeff': -0.3333333333333333, 'x': 2.0, 'y': -3.0, 'z': 1.0})

In [313]:
td1 ** td2  # 3^(-0.5*x*1/(y^3))

{'$coeff': -0.3333333333333333, 'x': 2.0, 'y': -3.0, 'z': 1.0}

In [314]:
print(td1 ** td2)

-0.3333333333333333*x^2*1/(y^3)*z


In [325]:
td3 = TermDict(3)
td4 = TermDict(2)
x = Variable('x')
y = Variable('y')
z = Variable('z')
td4[x] = 1
td4[y] = 1
xy_9 = td3 * td4

In [326]:

xy_9.calculate_variable({x:3, y:4, z:2}), xy_9, str(xy_9)

(({'$coeff': 72}, 'SUCCESS'), {'$coeff': 6, 'x': 1, 'y': 1}, '6*x*y')

In [302]:
x = Variable('x')
type(x), isinstance(x, Variable)
for key in td1.keys():
    print(type(key), isinstance(key, Variable))

<class 'str'> False
<class '__main__.Variable'> True


In [306]:
td1 = TermDict(3)
x = Variable('x')
td1[x] = x
td1.calculate_variable({'x':3.0})

({'$coeff': 81.0}, 'SUCCESS')

In [281]:
str(xy_9), str(xy_3), td3, td4

('9^(x*y)', '3^(x*y)', {'$coeff': 3}, {'$coeff': 1, 'x': 1, 'y': 1})

In [282]:
xy_9 + xy_3 + 3 ** td4

9^(x*y) {{'$coeff': 1, {'$coeff': 3}: {'$coeff': 1, 'x': 1, 'y': 1}}: 1, '$const': 0.0}
not in
3^(x*y) {{'$coeff': 1, {'$coeff': 3}: {'$coeff': 1, 'x': 1, 'y': 1}}: 1, '$const': 0.0, {'$coeff': 1, {'$coeff': 9}: {'$coeff': 1.0, 'x': 1, 'y': 1}}: 1}
in


{{'$coeff': 1, {'$coeff': 3}: {'$coeff': 1, 'x': 1, 'y': 1}}: 2,
 '$const': 0.0,
 {'$coeff': 1, {'$coeff': 9}: {'$coeff': 1.0, 'x': 1, 'y': 1}}: 1}

In [283]:
str(3 ** td4)

'3^(x*y)'

In [284]:
td4.get_str_without_coeff(), (-td4).get_str_without_coeff(), td4, -td4

('x*y', 'x*y', {'$coeff': 1, 'x': 1, 'y': 1}, {'$coeff': -1.0, 'x': 1, 'y': 1})

In [285]:
hash(td4.get_str_without_coeff()), hash((-td4).get_str_without_coeff())

(-1637283821803217022, -1637283821803217022)

In [286]:
td5 = td4 - td4

x*y {{'$coeff': -1.0, 'x': 1, 'y': 1}: -1.0, '$const': 0.0}
in


In [287]:
td5

{'$const': 0.0}

In [274]:
td5.keys()

dict_keys([{'$coeff': -1.0, 'x': 1, 'y': 1}, '$const', {'$coeff': 1, 'x': 1, 'y': 1}])

True